<a href="https://colab.research.google.com/github/sznajdr/sumpdf/blob/main/summarization_token_batching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title define model, text file link

#@markdown <font color="orange"> If out of CUDA memory, try the 
#@markdown smaller model `pszemraj/led-base-book-summary`

hf_tag = "pszemraj/led-base-book-summary" #@param ["pszemraj/led-large-book-summary", "pszemraj/led-base-book-summary"] {allow-input: true}

example_url = "https://www.dropbox.com/s/trm1xb6rdjdkgt9/mkdl-An%20Introduction%20to%20Deep%20Reinforcement%20Learning.txt?dl=1" #@param {type:"string"}

# setup

In [ ]:
#@title GPU info
#@markdown - usage of the model requires GPU, go to Runtime -> Change runtime type if needed
!nvidia-smi

Thu Sep  8 13:21:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@markdown add auto-Colab formatting with `IPython.display`
from IPython.display import HTML, display
# colab formatting
def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )

get_ipython().events.register("pre_run_cell", set_css)

In [ ]:
!pip install -U datasets transformers ninja -q
!pip install -U sentencepiece -q
!pip install clean-text[gpl] -q

     |████████████████████████████████| 365 kB 13.5 MB/s 
     |████████████████████████████████| 4.7 MB 33.8 MB/s 
     |████████████████████████████████| 108 kB 65.2 MB/s 
     |████████████████████████████████| 120 kB 72.0 MB/s 
     |████████████████████████████████| 212 kB 60.4 MB/s 
     |████████████████████████████████| 115 kB 52.8 MB/s 
     |████████████████████████████████| 127 kB 19.6 MB/s 
     |████████████████████████████████| 6.6 MB 45.0 MB/s 
     |████████████████████████████████| 1.3 MB 17.0 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 175 kB 28.2 MB/s 
     |████████████████████████████████| 235 kB 58.7 MB/s 


In [ ]:
#@title packages
!pip install Sentencepiece
!pip install ipywidgets
!pip install pdfplumber transformers
import nltk
nltk.download('punkt')
from IPython.display import clear_output
clear_output(wait=True)
print("Everything installed.")

Everything installed.


In [ ]:
from transformers import pipeline
import torch
from cleantext import clean
from pathlib import Path

_device = 0 if torch.cuda.is_available() else -1


In [ ]:
#@markdown setup logging
import logging
from pathlib import Path
das_logfile = Path.cwd() / "summarize_tokenbatches.log"

logging.basicConfig(
    level=logging.INFO,
    filename=das_logfile,
    format="%(asctime)s %(levelname)s %(message)s",
    datefmt="%m/%d/%Y %I:%M:%S",
)
print(f'logfile is at:\n\n{das_logfile}')

logfile is at:

/content/summarize_tokenbatches.log


In [ ]:
import requests
import re

#@markdown define `get_filename`

def get_filename(url:str):
    """
    Parses a URL string to find the filename of the file that it downloads.
    """
    # get the last part of the url
    url_split = url.split('/')
    last_part = url_split[-1]
    # replace "?dl=1"
    last_part = last_part.replace('?dl=1', '')

    if '.' in last_part:
        # if the last part is a file name, return it

        suffix = last_part.split('.', maxsplit=1)[-1]
        file_head = last_part.split('.', maxsplit=1)[0].replace("%", " ")
        # replace all non-alphanumeric or whitespace chars in file stem
        file_stem = re.sub(r'[^\w\s]', '', file_head)
        # replace all whitespace chars in file stem
        file_stem = re.sub(r'\s+', '_', file_stem)
        return file_stem + '.' + suffix

    
    # replace all non-alphanumeric or whitespace chars in file stem
    file_stem = re.sub(r'[^\w\s]', '', last_part)
    # replace all whitespace chars in file stem
    file_stem = re.sub(r'\s+', '_', file_stem)
    
    # get the file extension
    r = requests.get(url, stream=True)
    content_type = r.headers['content-type']
    file_extension = content_type.split('/')[-1]
    
    # return the file name
    return file_stem + '.' + file_extension

## load model and tokenizer

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(
    hf_tag, 
).to('cuda')

Some weights of LEDForConditionalGeneration were not initialized from the model checkpoint at pszemraj/led-large-book-summary and are newly initialized: ['led.decoder.embed_tokens.weight', 'led.encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    hf_tag, 
)


# Functions & Params



In [ ]:
#@markdown `get_timestamp(detailed=False)`

from datetime import datetime

def get_timestamp(detailed=False):
    """
    get_timestamp - returns a timestamp in string format

    detailed : bool, optional, default False, if True, returns a timestamp with seconds
    """
    if detailed:
        return datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    else:
        return datetime.now().strftime("%b-%d-%Y")

get_timestamp(True)

'2022-09-08_13-25-38'

### parameters


In [ ]:
#@markdown <font color="orange"> decrease `token_batch_length` if running OOM
token_batch_length = 16384 #@param ["16384", "8192", "4096", "3072", "2048"] {type:"raw"}
batch_stride = 20 #@param {type:"integer"}

session_settings = {}
session_settings['token_batch_length'] = token_batch_length
session_settings['batch_stride'] = batch_stride



In [ ]:
#@title generation parameters

#@markdown - <font color="orange">  decrease `token_batch_length` if running OOM
#@markdown - <font color="orange"> decrease `number_beams` if running OOM

number_beams = 16 #@param ["16", "12", "8", "4"] {type:"raw"}
min_length =  32#@param {type:"integer"}
max_len_ratio = 6.25 #@param {type:"slider", min:2, max:10, step:0.25}
length_penalty =  0.5#@param {type:"number"}

if token_batch_length > 8192 and number_beams > 8:
    logging.info(f'{number_beams} number_beams too high, reducing')
    number_beams = 8
settings = {
    'min_length':32,
    'max_length':int(token_batch_length//max_len_ratio),
    'no_repeat_ngram_size':3, 
    'encoder_no_repeat_ngram_size' :4,
    'repetition_penalty':3.7,
    'num_beams':number_beams,
    'length_penalty':length_penalty,
    'early_stopping':True,
    'do_sample':False,
}
logging.info(f"using textgen params:\n\n:{settings}")
session_settings['num_beams'] = number_beams
session_settings['length_penalty'] = length_penalty
session_settings['max_len_ratio'] = max_len_ratio

## define generation functions


In [ ]:
#@markdown single-shot fns
#@markdown - def `generate_answer()`

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]



def generate_answer(batch,**kwargs):

    inputs_dict = tokenizer(batch["text"], 
                            padding="max_length", max_length=16384, 
                            return_tensors="pt", 
                            truncation=True,
                            add_special_tokens =False,
                            )
    
    input_ids = inputs_dict.input_ids.to("cuda")
    attention_mask = inputs_dict.attention_mask.to("cuda")
    print(attention_mask, attention_mask.size())
    global_attention_mask = torch.zeros_like(attention_mask)
    # put global attention on <s> token
    global_attention_mask[:, 0] = 1

    predicted_abstract_ids = model.generate(
            input_ids, 
            attention_mask=attention_mask, 
            global_attention_mask=global_attention_mask, 
            **kwargs
        )
    batch["summary"] = tokenizer.batch_decode(predicted_abstract_ids, 
                                                skip_special_tokens=True,
                                                remove_invalid_values=True,
                                                )
    return batch

In [ ]:
# @title token batch summarization

#@markdown - def `summarize_and_score(ids, mask, **kwargs)`
#@markdown - def `summarize_via_tokenbatches(input_text:str, batch_length=8192, batch_stride=16, **kwargs,)`
def summarize_and_score(ids, mask, **kwargs):


    ids = ids[None, :]
    mask = mask[None, :]
    
    input_ids = ids.to("cuda")
    attention_mask = mask.to("cuda")
    global_attention_mask = torch.zeros_like(attention_mask)
    # put global attention on <s> token
    global_attention_mask[:, 0] = 1

    summary_pred_ids = model.generate(
            input_ids, 
            attention_mask=attention_mask, 
            global_attention_mask=global_attention_mask, 
            output_scores=True,
            return_dict_in_generate=True,
            **kwargs
        )
    summary = tokenizer.batch_decode(
                summary_pred_ids.sequences, 
                skip_special_tokens=True,
                remove_invalid_values=True,
            )
    score = round(summary_pred_ids.sequences_scores.cpu().numpy()[0], 4)
    
    return summary, score
    
def summarize_via_tokenbatches(
        input_text:str,
        batch_length=8192,
        batch_stride=16,
        **kwargs,
    ):
    
    encoded_input = tokenizer(
                        input_text, 
                        padding='max_length', 
                        truncation=True,
                        max_length=batch_length, 
                        stride=batch_stride,
                        return_overflowing_tokens=True,
                        add_special_tokens =False,
                        return_tensors='pt',
                    )
    
    in_id_arr, att_arr = encoded_input.input_ids, encoded_input.attention_mask
    gen_summaries = []

    pbar = tqdm(total=len(in_id_arr))

    for _id, _mask in zip(in_id_arr, att_arr):

        result, score = summarize_and_score(
            ids=_id, 
            mask=_mask, 
            **kwargs,
        )
        score = round(float(score),4)
        _sum = {
            "input_tokens":_id,
            "summary":result,
            "summary_score":score,
        }
        gen_summaries.append(_sum)
        print(f"\t{result[0]}\nScore:\t{score}")
        pbar.update()

    pbar.close()

    return gen_summaries

---


# summarize - single file

In [ ]:
#@title upload
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  with open('file.pdf', 'wb') as f:
    f.write(uploaded[filename])
    from IPython.display import clear_output
    clear_output(wait=True)
print("File uploaded successfully!")

File uploaded successfully!


In [ ]:
#@title seiten auswählen
import pdfplumber

filename = 'file.pdf'
start_page = int(input("Enter the start page number: "))
end_page = int(input("Enter the end page number: "))
output_filename = input("Enter the output filename: ")

if not output_filename.endswith('.txt'):
    output_filename += '.txt'

with pdfplumber.open(filename) as pdf:
    all_pages_text = ''
    for i, page in enumerate(pdf.pages):
        if i+1 >= start_page and i+1 <= end_page:
            extracted_text = page.extract_text()
            all_pages_text += extracted_text
            print(f"Extracting text from page {i+1}...")
            from IPython.display import clear_output
            clear_output(wait=True)
    with open(output_filename, 'w') as f:
        f.write(all_pages_text)
    print("Text extraction complete.")

Text extraction complete.


In [ ]:
#@title text extraction
import pdfplumber

filename = 'file.pdf'

with pdfplumber.open(filename) as pdf:
    with open('output.txt', 'w') as output_file:
        for i, page in enumerate(pdf.pages):
            extracted_text = page.extract_text()
            print(f"Extracting text from page {i+1}...")
            output_file.write(extracted_text + '\n')


In [ ]:
#@markdown read in single file text as `long_text`
with open("output.txt", 'r', errors='ignore') as f:
    raw_text = f.read()

long_text = clean(raw_text, lower=False)
logging.info(f"removed {len(long_text) - len(raw_text)} chars via cleaning")
batch = {}
batch['text'] = long_text


encoded_input = tokenizer(
    long_text, 
    padding='max_length', 
    truncation=True,
    max_length=token_batch_length, 
    stride=batch_stride,
    return_overflowing_tokens=True,
    add_special_tokens =False,
    return_tensors='pt',
)

In [ ]:
#@markdown summaries for funsies!

_summaries = summarize_via_tokenbatches(
    long_text,
    batch_length=token_batch_length,
    batch_stride=batch_stride,
    **settings,
)


  0%|          | 0/3 [00:00<?, ?it/s]

	In this chapter, the author explains how reinforcement learning is used to train artificial intelligence agents. He uses examples from games such as Super Mario Bros. and chess to demonstrate how an agent learns by interacting with its environment and receiving rewards. From this interaction, the agent learns how to perform actions in a game, gain experience, and then move to the next state. The agent always seeks to maximize the expected return, which is the sum of all the actions taken.
Score:	-5.546
	Now that we've learned about rewards, it's time to get down to business. The reward is "the only feedback" the agent gets in RL--it's the only way for him to know if he/she got rewarded for taking a certain action. In order to make this process as smooth as possible, the authors divide each step into two parts: the episodic tasks and the continuous tasks. Episodic tasks are tasks that have a starting place and an end point and can begin and end with the same state of the game. For exam

In [ ]:
#@markdown write the `_summaries` var to a `.txt`
sum_text = [s["summary"][0] for s in _summaries]
sum_scores = [f"\n - {round(s['summary_score'],4)}" for s in _summaries]
scores_text = "\n".join(sum_scores)
full_summary = "\n\t".join(sum_text)
_outpath = f"SUM_{example_path.name}"

with open(
    _outpath,
    "w",
) as fo:
    fo.writelines(full_summary)
    fo.write("\n" * 3)
    fo.write(f"\n\nSection Scores for {example_path.name}:\n")
    fo.writelines(scores_text)
    fo.write("\n\n---\n")


In [ ]:
#@markdown print out the summarized output!
!cat $_outpath

In this chapter, the author explains how reinforcement learning is used to train artificial intelligence agents. He uses examples from games such as Super Mario Bros. and chess to demonstrate how an agent learns by interacting with its environment and receiving rewards. From this interaction, the agent learns how to perform actions in a game, gain experience, and then move to the next state. The agent always seeks to maximize the expected return, which is the sum of all the actions taken.
	Now that we've learned about rewards, it's time to get down to business. The reward is "the only feedback" the agent gets in RL--it's the only way for him to know if he/she got rewarded for taking a certain action. In order to make this process as smooth as possible, the authors divide each step into two parts: the episodic tasks and the continuous tasks. Episodic tasks are tasks that have a starting place and an end point and can begin and end with the same state of the game. For example, an agent e

<font color="salmon"> note that the middle summary, arguably the worst quality, received the most-negative score. </font>


---


# summarize - txt directory

> demonstrate use case of iterating over all text files in a directory


## load

In [ ]:
zip_url = "https://www.dropbox.com/sh/c03o2gpcvh6v3yz/AACaxe00trjjuV4Zmdpn1JtOa?dl=1" #@param {type:"string"}
target_path = "/content/source-text" #@param {type:"string"}
target_path = Path(target_path)
zip_fname = 'temp.zip'

!rm -r $target_path
!rm $zip_fname

!wget $zip_url -O $zip_fname
!unzip -q $zip_fname -d $target_path

In [ ]:
#@markdown importing txt
import random

txt_files = [f for f in target_path.iterdir() if f.is_file() and f.suffix == '.txt']
output_dir = target_path.parent / "summarized-text"
output_dir.mkdir(exist_ok=True)

random.SystemRandom().shuffle(txt_files)
txt_files


[PosixPath('/content/source-text/mkdl-Introducing Hugging Face for Education.txt'),
 PosixPath('/content/source-text/mkdl-An Introduction to Q-Learning Part 1.txt'),
 PosixPath('/content/source-text/mkdl-Introducing Decision Transformers.txt')]

## summarize

- runs model in a loop over each text file in a directory. 
- FOR each text file: break into batches of `token_batch_length` tokens, which overlap with the `batch_stride` defined earlier.



In [ ]:
#@markdown summarize loop

for f in tqdm(txt_files, total=len(txt_files)):

    with open(f, 'r', encoding='utf-8', errors='ignore') as fi:
        text = clean(fi.read(), lower=False)
    print(f"\nNow summarizing:\t{f.name}")
    _summaries = summarize_via_tokenbatches(
                    text,
                    batch_length=token_batch_length,
                    batch_stride=batch_stride,
                    **settings,
                )
    sum_text = [s['summary'][0] for s in _summaries]
    sum_scores = [f"\n - {round(s['summary_score'],4)}" for s in _summaries]
    scores_text = "\n".join(sum_scores)
    full_summary = "\n\t".join(sum_text)
    with open(output_dir / f"SUM_{f.name}", 'w', ) as fo:
        fo.writelines(full_summary)
    with open(output_dir / f"SESSION_SCORES.log", 'a', ) as fo:
    
        fo.write("\n"*3)
        fo.write(f"\n\nSection Scores for {f.name}:\n")
        fo.writelines(scores_text)
        fo.write("\n\n---\n")

  0%|          | 0/3 [00:00<?, ?it/s]


Now summarizing:	mkdl-Introducing Hugging Face for Education.txt


  0%|          | 0/1 [00:00<?, ?it/s]

	As the use of machine learning increases, so does the need for better education. Hugging Face aims to democratize this process by making it free and open source. In order to do this, they've created a number of courses and workshops that cover a broad range of topics from natural language processing to deep reinforcement learning to Transformers. They're also hosting meetups and events where people can share and experiment with their models. Finally, they want to empower teachers by giving them free tools and resources so they can teach their students how to use machine learning in their classroom.
Score:	-5.2706

Now summarizing:	mkdl-An Introduction to Q-Learning Part 1.txt


  0%|          | 0/3 [00:00<?, ?it/s]

	In this final part of the class, the professor teaches his students about reinforcement learning and introduces some of the most important concepts in the field. In this chapter, he covers two different types of reinforcement learning: value-oriented methods and methodologies that focus on finding the "optimal policy," i.e., the "best expected cumulative reward." The goal of an RL agent is to find the "an optimal policy p" that maximizes the expected gain from any given set of actions. To do this, the agent must be able to predict what actions will lead to the greatest gain, either by observing the current state of the world or by predicting future outcomes based on past results. Using a value function called the State-Value Function, the agent determines the expected return from taking action at each state. However, this can be a time-consuming task since it requires calculation of all the possible rewards an agent could possibly receive from a given state. Fortunately, the bellman e

  0%|          | 0/2 [00:00<?, ?it/s]

	In this paper, the authors introduce a new approach to reinforcement learning, called Decision Transformers. Instead of training an agent on a dataset and then comparing it to a simulator, they instead use data from real-world demonstrations to train the agent. They claim that this approach can be more appropriate for offline applications because the agent does not need to interact with the outside world as much. Their approach uses a Decision Transformer, which allows them to train an agent in a laboratory without ever having to leave the house.
Score:	-6.0189
	Next, we need some target returns for the neural network, to help us control the model's performance in a real-world scenario where we want to fine-tune an opponent bot's performance against our own. The authors of the paper demonstrate how they use a weighted average of the state-mean and standard error across multiple datasets to predict the expected return from each neuralnetwork agent. This is called "the power of offline 

---


## export summarized dir

- this downloads a `.zip` of the summarized text files.



In [ ]:
output_zip_tag = "hf_demo" #@param {type:"string"}
output_zip = f"summarized_textdir_{output_zip_tag}.zip"
output_zip = Path(output_zip)
output_zip = Path(output_zip.stem + get_timestamp() + '.zip')
!zip -r -q -9 $output_zip /content/summarized-text

In [ ]:
from google.colab import files
files.download(output_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>